Calculate Metrics
=================
Calculates a number of metrics for an already existing street graph with lanes before and after

Prerequisites: Prepare the street graph and rebuild it

In [ ]:
import snman
from snman.constants import *

PERIMETER = '_debug'

# Set these paths according to your own setup
data_directory = 'C:/Users/lballo/polybox/Research/SNMan/SNMan Shared/data_v2/'
inputs_path = data_directory + 'inputs/'
outputs_path = data_directory + 'outputs/' + PERIMETER + '/'
metrics_path = data_directory + 'metrics/' + '_debug' + '/'

Loading data
------------
Loads the street graph and measurement regions

In [ ]:
print('Load street graph')
G = snman.io.load_street_graph(outputs_path + 'street_graph_edges.gpkg', outputs_path + 'street_graph_nodes.gpkg')

print('Load measurement regions')
measurement_regions_gdf = snman.io.load_measurement_regions(inputs_path + 'measurement_regions/measurement_regions.gpkg',)

Calculating metrics
-------------------
This step will calculate a series of network metrics for lane graphs of different modes before and after rebuilding.
Please note that all calculations only work with strongly connected graphs. Therefore, all lane graphs will be automatically clipped only to the largest strongly connected component. Set *plot_scc=True* to show which parts of the graphs are removed (nodes are colored by their strongly connected component). Most likely, there will always be a few nodes that must be removed. But if they are many, then the results will be distorted.

For better accuracy, it's recommended that you adjust the perimeter of your network such that there are no nodes outside the largest connected component.

In [ ]:
# filter out the highways
G_filtered = snman.street_graph.filter_by_hierarchy(G, snman.hierarchy.HIERARCHIES.difference(snman.hierarchy.HIGHWAY))

# calculate the metrics
metrics = snman.stats.network_metrics_for_all_measurement_regions(G_filtered, measurement_regions_gdf, plot_scc=True)

Viewing the metrics
-------------------
The metrics are saved as a dict of DataFrames, one for each measurement region.
See the docustring of snman.lane_graph.calculate_stats for an explanation of the measures.

In [ ]:
metrics['zurich_city'].to_excel(metrics_path + 'metrics_zurich_city.xlsx', index=True)